In [1]:
import pandas as pd
from nltk import tokenize
import torch
import os
import time
import tensorflow as tf
import numpy as np
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, CallbackContext
import re
from dicts import terms_dict # словарь терминов и жаргонизмов из профессиональной области

### 0. Заменяем часть слов по словарю
#### Некоторые термины, жаргонные названия и аббревиатуры меняем на "общепринятый" язык, которые использовался в обучающих данных и контексте для берта.

In [2]:
def sub_by_dict(text, dictionary):
    for word in text.split():
        cleared_word = re.sub('[.!?&,]', '', word) # за некоторомы словами следуют знаки препинания
        if cleared_word in dictionary.keys():
            text = text.replace(cleared_word, dictionary[cleared_word])
    return text

### 1. Ищем ответы на вопросы из базы
##### Используем эмбединги из БЕРТа, чтобы обучить сеточку-классификатор. 

In [3]:
from transformers import AutoTokenizer, TFAutoModel

In [4]:
qa_threshold_top = 0.85
qa_threshold_bottom = 0.65

In [5]:
#model_faq = train_model(model_config)
model_faq = tf.keras.models.load_model('models/classificator.h5')

In [6]:
tokenizer_bert = AutoTokenizer.from_pretrained('Geotrend/bert-base-ru-cased')
model_bert = TFAutoModel.from_pretrained('Geotrend/bert-base-ru-cased')

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
df = pd.read_excel('data/QAfinal.xls', usecols=['Answer', 'Class', 'Recommend'])
df.drop_duplicates(subset='Class', keep="last", inplace=True)

In [8]:
def get_recommend(question_class):
    recs = df.loc[df.Class == question_class]['Recommend'].values[0].split('* ')
    return recs

In [9]:
def get_qa_answer(text):
    
    question = tokenizer_bert(text,
                max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    question_out =  model_bert(question)
    question_emb = question_out.pooler_output
    pred = model_faq.predict(question_emb)
    score = (pred[0][np.argmax(pred)])

    answer = df.loc[df.Class == np.argmax(pred)]['Answer'].values[0]
    
    recommendation = get_recommend(np.argmax(pred))
    
    return answer, score, recommendation

In [10]:
#get_qa_answer('вы проводите курсы по аноль?')

('РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп.',
 0.72599155,
 ['Какая цена у сметной программы?', 'сколько стоит сопровождение?'])

In [11]:
#get_qa_answer('мне нужно обучение')

('РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп.',
 0.9957547,
 ['Какая цена у сметной программы?', 'сколько стоит сопровождение?'])

### 2. Пробуем найти ответ в моделе question-answering
##### опять таки на основе БЕРТа

In [12]:
from transformers import pipeline

In [13]:
bert_threshold = 0.01

In [14]:
# qa_pipeline = pipeline(
#     "question-answering",
#     model="mrm8488/bert-multi-cased-finetuned-xquadv1",
#     tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1"
# )

In [15]:
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp",
    tokenizer="mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp"
)

In [16]:
context = '''
РЦЦС - Региональный центр ценообразования в строительстве.
Мы оказываем услуги по сопровождению сметных программ, составлению сметной документации, разработке нормативов и негосударственной экспертизе смет.
РЦЦС занимается сопровождением сметных программ, составлением смет, разработкой нормативов и негосударственной экспертизой смет.
Разработчик программы А-ноль - ГК "Инфострой", Санкт-Петербург.
Разработчик Мурманской ТЕР - РЦЦС по Мурманской области.
У нас вы можете купить и обновить сметную программу, А-ноль и Грандсмету.
В состав обслуживания входит обновление сборников цен, сборников индексов и линия консультаций.
Сборник индексов содержит построчные индексы по элементам затрат.
Сборник цен и сборник индексов распространяется в составе квартального обновления.
ТЕР это территориальные единичные расценки.
ФЕР это федеральные единичные расценки.
ФСНБ - это федеральная сметно-нормативная база, состоит из ФЕР и ГЭСН.
ГЭСН это государственные элементные сметные нормы.
Обновление версии А-ноль входит в квартальное обслуживание.
Обновление версии Гранд осуществляется в виде годовой подписки стоимостью 25000 рублей.
Вы можете обратиться на линию консультаций по номеру 400-507.
Если у вас возникли вопросы по программе - обратитесь на нашу линию консультаций.
Если вам нужна техническая поддержке - позвоните по номеру 400-500, доб 224.
При возниковении технических сложностей или ошибок можно обратиться в техническую поддержку.
Не запускается программа или появляются ошибки - обратитесь к тех поддержку.
Если нужна консультация по А-ноль или Гранд, вы можете обратиться на линию консультаций 400-507.
Телефон техподдержки 400-500, доб 224.
'''

In [17]:
context_splitted = context.split('.')

n = 0
sent_dict = {-1: None}

for sent in context_splitted:
    sent_dict[n] = sent
    n += len(sent)

In [18]:
def get_full_bert_answer(ans_position):
    k = -1
    for i in range(len(sent_dict.keys())):
        if list(sent_dict.keys())[i] < ans_position and list(sent_dict.keys())[i+1] > ans_position:
            k = list(sent_dict.keys())[i]
            
    return sent_dict[k]

In [19]:
# test_question = 'как проконсультироваться'

# result =  qa_pipeline({
#                 'context': context,
#                 'question': test_question
#             })

# result

In [20]:
# get_full_bert_answer(result['start'])

### 3. Если ничего не подошло - пересылаем в rudialogpt

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [22]:
model_gpt = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")

In [23]:
tokenizer = AutoTokenizer.from_pretrained('Grossmend/rudialogpt3_medium_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
def get_length_param(text: str) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [25]:
def get_answer_gpt(text):
    input_user = text
    new_user_input_ids = tokenizer.encode(f"|0|{get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|", return_tensors="pt")
    
    chat_history_ids = model_gpt.generate(
        new_user_input_ids,
        num_return_sequences=1,
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature = 0.6,
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
        device='cpu',
    )
    
    answer = tokenizer.decode(chat_history_ids[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    return answer
    

In [26]:
get_answer_gpt('Привет буфет')

'Свежее'

### 4. Последовательно передаем вопрос через наши модели

In [27]:
##### разбиваем текст на предложения и отвечаем на каждое предложение
def get_full_answer(request):

    questions = request
    final_response = ''
    for question in tokenize.sent_tokenize(questions):
        question_for_bert = question
        question = sub_by_dict(question, terms_dict) # + "?"
        
        qa_score = 0
        bert_score = 0
        recs = [0,0]
        if len(question.split()) > 2:
            # ответ QA
            print(question)
            answer , qa_score, recs = get_qa_answer(question)
            print(answer, qa_score)
        
        
        
        # если у ответа плохой скор - перенаправляем вопрос в берт
        if qa_score < qa_threshold_bottom:
            result =  qa_pipeline({
                            'context': context,
                            'question': question_for_bert
                            })
            print(result)
            position = result['start']
            bert_score = result['score']
            #print(position)
            if position != -1 and bert_score > bert_threshold:
                answer = get_full_bert_answer(position)
            else:
                answer = get_answer_gpt(question) #+ ' gpt'
    
    
    
        final_response += answer + '\n'
        print(question, '* |', answer,'\n QA: ', qa_score, ' bert: ', bert_score, '\n')
    return final_response, qa_score, recs

In [28]:
testing = 'Мне нужна сметная программа. Сколько стоит обновление? воняешь бедностью? какие услуги вы оказываете'

In [29]:
get_full_answer(testing)

Мне нужна сметная программа.
Стоимость ПК "А-ноль" 43 000 рублей. Стоимость ПК "Грандсмета" 29 000 рублей. 0.5703831
{'score': 0.05776384100317955, 'start': 208, 'end': 212, 'answer': 'РЦЦС'}
Мне нужна сметная программа. * | 
РЦЦС занимается сопровождением сметных программ, составлением смет, разработкой нормативов и негосударственной экспертизой смет 
 QA:  0.5703831  bert:  0.05776384100317955 

Сколько стоит обновление?
Стоимость обслуживания составляет 25 200 рублей в квартал за основное рабочее место, 12 600 за дополнительное. 0.9837204
Сколько стоит обновление? * | Стоимость обслуживания составляет 25 200 рублей в квартал за основное рабочее место, 12 600 за дополнительное. 
 QA:  0.9837204  bert:  0 

{'score': 6.734969065291807e-05, 'start': 8, 'end': 58, 'answer': 'Региональный центр ценообразования в строительстве'}
воняешь бедностью? * | это не бедность, а состояние души. 
 QA:  0  bert:  6.734969065291807e-05 

какие услуги вы оказываете
Цена на ПК "А-ноль" составляет 43 00

('\nРЦЦС занимается сопровождением сметных программ, составлением смет, разработкой нормативов и негосударственной экспертизой смет\nСтоимость обслуживания составляет 25 200 рублей в квартал за основное рабочее место, 12 600 за дополнительное.\nэто не бедность, а состояние души.\nПомощь в выборе, консультации, помощь в оформлении документов.\n',
 0.6072348,
 ['я не умеют работать в А-ноль', 'Какая стоимость обслуживания?'])

### Часть с ботом

In [30]:
def chat_log(question, user_name, answer):
    writepath = './messages/' + user_name + '.txt'
    mode = 'a' if os.path.exists(writepath) else 'w'
    with open(writepath, mode) as f:
        f.write(time.ctime() + '\n' + user_name + ': ' + question + '\n' + 'Bot: ' + answer + '\n\n')

In [31]:
#Настройки 
updater = Updater(token='########') # Токен API к Telegram
dispatcher = updater.dispatcher

In [32]:
# Обработка команд
def startCommand(update, bot):
    bot.bot.send_message(chat_id=update.effective_chat.id, text="Добрый день. Я - чат-бот РЦЦС. Могу проконсультировать Вас по стоимости сметных программ и нормативов")
    
    
def textMessage(update, bot):

    # формируем строку для логов и передачу вопроса в модель
    question = update.message.text
    user_name =  update.message.chat.username
    if user_name is None:
        user_name = update.message.chat.last_name + ' ' + update.message.chat.first_name
        
    answer, qa_score, recs = get_full_answer(question)    
    chat_log(question, user_name, answer)    
    
    print(recs)
    if qa_score < qa_threshold_top and recs!=[0,0]:
#         keyboard = [
#                 [
#                     InlineKeyboardButton(recs[0], callback_data=recs[0]),
#                     InlineKeyboardButton(recs[1], callback_data=recs[1])
#                 ],
#                 [
#                     InlineKeyboardButton("Отмена", callback_data='Отмена')
#                 ]
#             ]
        keyboard = [
                [InlineKeyboardButton(recs[0], callback_data=recs[0])],
                [InlineKeyboardButton(recs[1], callback_data=recs[1])],
                [InlineKeyboardButton("Отмена", callback_data='Отмена')]
            ]
        
         
    
        reply_markup = InlineKeyboardMarkup(keyboard)   
        
        bot.bot.send_message(chat_id=update.effective_chat.id, text=answer,reply_markup=reply_markup)
    else:
        bot.bot.send_message(chat_id=update.effective_chat.id, text=answer)
    
    
def button(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    
    # This will define which button the user tapped on (from what you assigned to "callback_data". As I assigned them "1" and "2"):
    choice = query.data
    
    
    user_name =  query.from_user.username
    if user_name is None:
        user_name = query.from_user.last_name + ' ' + query.from_user.first_name
        
    print(query.chat_instance)
    print(query.from_user)
    
    # Now u can define what choice ("callback_data") do what like this:
    if choice == 'Отмена':
        chat_log(choice + ' INLINE REQUEST', user_name, choice) 
        query.edit_message_reply_markup()
    else:
        answer, _, _ = get_full_answer(choice)  
        print(choice, user_name, answer)
        chat_log(choice, user_name, answer)    
        #query.edit_message_text(text=answer)
        query.edit_message_reply_markup()
        query.bot.send_message(chat_id=update.effective_chat.id, text=answer)



In [ ]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)

updater.dispatcher.add_handler(CallbackQueryHandler(button))

# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)

# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

<ipython-input-34-f128d0fc0b06>:12: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True)
